# NCALM

This notebook illustrates search, directly loading DEMs, and downloading DEMs and LPCs for local processing

| Dataset                      | Alias      | Type      | Start      | End        | Extent         | Source                                                                      |
| ---------------------------- | ---------- | --------- | ---------- | ---------- | -------------- | --------------------------------------------------------------------------- |
| NCALM LiDAR                  | ncalm      | LiDAR     | 2003-05-15 |            | US Territories | [NCALM](https://calm.geo.berkeley.edu/ncalm/dtc.html)                       |

In [ ]:
import coincident
import geopandas as gpd
from shapely.geometry import box

## Search 

We'll search for data in Washington State

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/WA/shape.geojson"
)
aoi.plot();

In [ ]:
gf_ncalm = coincident.search.search(
    dataset="ncalm", intersects=aoi, datetime=["2018-09-19"]
)
print(f"Found {len(gf_ncalm)} NOAA scenes")
gf_ncalm.head(3)

## Get Data

```{warning}
The NCALM DEMs are not tiled, so reading in the data and downloading takes a longer time
```

### Stream 

First we'll show how to stream a gridded DEM directly into Xarray

In [ ]:
buffer_size = 0.01
centroid = gf_ncalm.centroid
mini_aoi = gpd.GeoDataFrame(
    geometry=[
        box(
            centroid.x - buffer_size,
            centroid.y - buffer_size,
            centroid.x + buffer_size,
            centroid.y + buffer_size,
        )
    ],
    crs="EPSG:4326",
)

m = gf_ncalm.explore(color="black")
mini_aoi.explore(m=m, color="magenta")

In [ ]:
%%time
da_ncalm_dtm = coincident.io.xarray.load_ncalm_dem(
    aoi=mini_aoi, product="dtm", dataset_id=gf_ncalm["name"].item()
)

In [ ]:
da_ncalm_dtm

In [ ]:
da_ncalm_dtm.coarsen(x=5, y=5, boundary="trim").mean().plot.imshow();

### Download 

```{note}
You will not see a download progress bar for downloading NCALM DEMs as opposed to USGS and NEON provider DEMs
```

#### DEM

In [ ]:
%%time

local_output_dir = "/tmp/ncalm/"

coincident.io.download.download_ncalm_dem(
    aoi=mini_aoi,
    dataset_id=gf_ncalm["name"].item(),
    product="dtm",
    output_dir=local_output_dir,
)

#### LPC Tiles

In [ ]:
%%time
gf_ncalm_lpc_tiles = coincident.io.download.fetch_lpc_tiles(
    aoi=mini_aoi, dataset_id=gf_ncalm.name.item(), provider="NCALM"
)

In [ ]:
gf_ncalm_lpc_tiles.head()

In [ ]:
m = gf_ncalm.explore(color="black")
gf_ncalm_lpc_tiles.explore(m=m, color="magenta")
mini_aoi.explore(m=m, color="black")